In [ ]:
# =====================================
# Mixed-Effects Model: 城市化 × 不平等交互 (按 Subcontinental 分组)
# =====================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
import matplotlib.pyplot as plt
import seaborn as sns
import os

# ==============================
# 1. 读取数据
# ==============================
file_path = r'C:\Users\Dell\Desktop\Global Urban\GID02_communities\Coastal_SUM_with_IRI_processed.xlsx'
df = pd.read_excel(file_path)

# ==============================
# 2. 列名标准化
# ==============================
df = df.rename(columns={
    'Per GDP': 'Per_GDP',
    'Urban ratio': 'Urban_ratio',
    'POP density': 'POP_density',
    'Floods_2020_pop_GID02': 'Floods_2020_pop',
    'Aging(60-)': 'Aging_60',
    'Young(0-10)': 'Young_0_10'
})

# ==============================
# 3. 循环次洲际区域绘图
# ==============================
output_dir = r'C:\Users\Dell\Desktop\Global Urban\Plots_Subcontinental'
os.makedirs(output_dir, exist_ok=True)

subcontinents = df['Subcontinental'].dropna().unique()

for sub in subcontinents:
    df_sub = df[df['Subcontinental'] == sub].copy()
    
    # ------------------------------
    # 数据清理
    # ------------------------------
    cols_needed = [
        'Floods_2020_Vulnerability', 'Infra_Resilience_Index', 'POP_2020_GID02',
        'Per_GDP', 'Inequality', 'Urban_ratio', 'POP_density', 'DEM', 'GID_0'
    ]
    df_sub = df_sub.dropna(subset=cols_needed)
    df_sub = df_sub[
        (df_sub['Floods_2020_Vulnerability'] > 0) &
        (df_sub['Infra_Resilience_Index'] > 0) &
        (df_sub['POP_2020_GID02'] > 0) &
        (df_sub['Per_GDP'] > 0) &
        (df_sub['POP_density'] > 0)
    ].copy()
    
    # ==============================
    # 计算 IFBI 指标
    # ==============================
    x = df_sub['Floods_2020_Vulnerability']
    y = df_sub['Infra_Resilience_Index']
    pop = df_sub['POP_2020_GID02']

    valid_mask = (x != 0) & (y != 0) & x.notna() & y.notna() & (pop > 0)
    df_sub = df_sub[valid_mask].copy()

    # 去除极端值（1%-99%）
    x_q_low, x_q_high = x.quantile([0.01, 0.99])
    y_q_low, y_q_high = y.quantile([0.01, 0.99])
    extreme_mask = (x >= x_q_low) & (x <= x_q_high) & (y >= y_q_low) & (y <= y_q_high)
    df_sub = df_sub[extreme_mask].copy()

    df_sub['IFBI'] = df_sub['Floods_2020_pop'] * df_sub['Floods_2020_Vulnerability'] 

    # ==============================
    # 变量转换与交互项
    # ==============================
    df_sub['log_IFBI'] = np.log(df_sub['IFBI'] + 1e-6)
    df_sub['log_Per_GDP'] = np.log(df_sub['Per_GDP'] + 1e-6)
    df_sub['InfraGap'] = 1 - df_sub['Infra_Resilience_Index']
    df_sub['Inequality_x_InfraGap'] = df_sub['Inequality'] * df_sub['InfraGap']
    df_sub['Urban_ratio_sq'] = df_sub['Urban_ratio'] ** 2
    df_sub['Urban_x_Ineq'] = df_sub['Urban_ratio'] * df_sub['Inequality']
    df_sub['GID_0'] = df_sub['GID_0'].astype(str)

    df_sub = df_sub.replace([np.inf, -np.inf], np.nan).dropna(subset=[
        'log_IFBI','log_Per_GDP','Inequality','Urban_ratio',
        'POP_density','DEM','Inequality_x_InfraGap','Urban_ratio_sq','Urban_x_Ineq'
    ])
    
    if len(df_sub) < 20:  # 样本太少跳过
        print(f"{sub} 样本太少，跳过绘图")
        continue

    # ==============================
    # 7. 混合效应模型
    # ==============================
    formula = (
        "log_IFBI ~ log_Per_GDP + Inequality + Urban_ratio + Urban_ratio_sq + "
        "POP_density + DEM + Inequality_x_InfraGap + Urban_x_Ineq"
    )
    md = mixedlm(formula, df_sub, groups=df_sub['GID_0'])
    mdf = md.fit(reml=False)
    print(f"===== {sub} =====")
    print(mdf.summary())

    # ==============================
    # 8. 可视化
    # ==============================
    urban_range = np.linspace(df_sub['Urban_ratio'].min(), df_sub['Urban_ratio'].max(), 100)
    ineq_levels = df_sub['Inequality'].quantile([0.1, 0.9]).values
    ineq_labels = ['Low Inequality (p25)', 'High Inequality (p75)']

    params = mdf.params
    cov_matrix = mdf.cov_params()
    pvalues = mdf.pvalues
    fixed_effects = [p for p in params.index if ("Group" not in p) and (p != "Group Var")]
    intercept_name = 'Intercept' if 'Intercept' in params.index else ('const' if 'const' in params.index else None)
    if intercept_name is not None:
        params_used = params[[intercept_name] + [p for p in fixed_effects if p != intercept_name]]
    else:
        params_used = params[fixed_effects]
    cov_used = cov_matrix.loc[params_used.index, params_used.index]

    plt.figure(figsize=(3, 2), constrained_layout=True)
    colors = ['#0072B2', '#D55E00']

    for ineq_val, label, color in zip(ineq_levels, ineq_labels, colors):
        pred_df = pd.DataFrame({
            'log_Per_GDP': df_sub['log_Per_GDP'].mean(),
            'Inequality': ineq_val,
            'Urban_ratio': urban_range,
            'Urban_ratio_sq': urban_range**2,
            'POP_density': df_sub['POP_density'].mean(),
            'DEM': df_sub['DEM'].mean(),
            'Inequality_x_InfraGap': ineq_val * df_sub['InfraGap'].mean(),
            'Urban_x_Ineq': urban_range * ineq_val,
        })
        design_df = pred_df[[c for c in pred_df.columns if c in params_used.index]].copy()
        if intercept_name is not None:
            design_df.insert(0, intercept_name, 1.0)
        design_df = design_df.loc[:, params_used.index]

        exog = design_df.values
        param_vals = params_used.values

        predicted = np.dot(exog, param_vals)
        se = np.sqrt(np.einsum('ij,jk,ik->i', exog, cov_used.values, exog))
        ci_upper = predicted + 1.96 * se
        ci_lower = predicted - 1.96 * se

        plt.plot(urban_range, predicted, color=color, linewidth=2.5, label=label)
        # plt.fill_between(urban_range, ci_lower, ci_upper, color=color, alpha=0.2)

    # --- 显著性标注 ---
    def sig_stars(p):
        if p < 0.01:
            return '***'
        elif p < 0.05:
            return '**'
        elif p < 0.1:
            return '*'
        else:
            return 'ns'

    # 坐标与风格
    plt.axvspan(0, 0.3, color='gray', alpha=0.1, label='Low urbanization')
    plt.axvspan(0.3, 0.7, color='gray', alpha=0.05, label='Medium')
    plt.axvspan(0.7, 1.0, color='gray', alpha=0.1, label='High')

    plt.xlabel("Urbanization Ratio", fontsize=12)
    plt.ylabel("Predicted CFR", fontsize=12)
    plt.title(f"{sub}: Coastal Urbanization × Inequality", fontsize=12)

    ax = plt.gca()
    ax.margins(x=0)
    for spine in ax.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(1.2)
    sns.despine()
    
    # # 图例
    # leg = plt.legend(loc='best', frameon=False, fontsize=14)

    # # 显著性标注放在图例下方
    # bbox = leg.get_window_extent()
    # inv = plt.gcf().transFigure.inverted()
    # bbox_fig = bbox.transformed(inv)
   
    # plt.grid(False)
    # plt.tight_layout()

    # ==============================
    # 保存图像
    # ==============================
    save_path = os.path.join(output_dir, f"{sub}_Urban_Inequality.tif")
    plt.savefig(
    save_path,
    dpi=300,
    format='tif',
    bbox_inches='tight',   # ⭐ 核心
    pad_inches=0.02        # ⭐ 极小安全边距
    )
    plt.close()
    print(f"✅ 已保存 {save_path}")


In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import mixedlm
from matplotlib import colors

# ==============================
# 0. 全局绘图参数
# ==============================
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 13,
    'axes.titlesize': 15,
    'xtick.labelsize': 11,
    'ytick.labelsize': 11
})

# ==============================
# 1. 读取数据
# ==============================
file_path = r'C:\Users\Dell\Desktop\Global Urban\GID02_communities\Coastal_SUM_with_IRI_processed.xlsx'
df = pd.read_excel(file_path)

# ==============================
# 2. 列名统一
# ==============================
df = df.rename(columns={
    'Per GDP': 'Per_GDP',
    'Urban ratio': 'Urban_ratio',
    'POP density': 'POP_density',
    'Floods_2020_pop_GID02': 'Floods_2020_pop',
    'Aging(60-)': 'Aging_60',
    'Young(0-10)': 'Young_0_10'
})

# ==============================
# 3. 基础清洗
# ==============================
x = df['Floods_2020_Vulnerability']
y = df['Infra_Resilience_Index']
pop = df['POP_2020_GID02']

df = df[
    (x > 0) & (y > 0) & (pop > 0) &
    x.notna() & y.notna()
].copy()

# 去除极端值（1%–99%）
df = df[
    df['Floods_2020_Vulnerability'].between(
        x.quantile(0.01), x.quantile(0.99)
    ) &
    df['Infra_Resilience_Index'].between(
        y.quantile(0.01), y.quantile(0.99)
    )
].copy()

# ==============================
# 4. 构建变量
# ==============================
df['IFBI'] = df['Floods_2020_pop'] * df['Floods_2020_Vulnerability']
df['log_IFBI'] = np.log(df['IFBI'] + 1e-6)
df['log_Per_GDP'] = np.log(df['Per_GDP'] + 1e-6)
df['GID_0'] = df['GID_0'].astype(str)
df['GID_2'] = df['GID_2'].astype(str)

df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[
    'log_IFBI', 'log_Per_GDP', 'Inequality',
    'Urban_ratio', 'POP_density', 'DEM',
    'Infra_Resilience_Index', 'Subcontinental'
])

# ==============================
# 5. MixedLM 模型公式
# ==============================
formula = (
    "log_IFBI ~ log_Per_GDP + Inequality + Urban_ratio + "
    "POP_density + DEM + Aging_60 + Young_0_10 + Infra_Resilience_Index"
)

# ==============================
# 6. 按次洲际建模
# ==============================
results = []

for region in sorted(df['Subcontinental'].unique()):
    df_r = df[df['Subcontinental'] == region].copy()

    if len(df_r) < 100 or df_r['GID_0'].nunique() < 3:
        continue

    try:
        md = mixedlm(formula, df_r, groups=df_r['GID_0'])
        mdf = md.fit(reml=False, disp=False)

        results.append({
            'Subcontinental': region,
            'coef': mdf.params['Inequality'],
            'pval': mdf.pvalues['Inequality']
        })

        print(f"✅ {region}: coef={mdf.params['Inequality']:.3f}")

    except Exception as e:
        print(f"❌ {region} failed: {e}")

res_df = pd.DataFrame(results)

# ==============================
# 7. 回归系数映射回社区
# ==============================
df_map = df.merge(
    res_df.rename(columns={'coef': 'Ineq_coef'}),
    on='Subcontinental',
    how='left'
)

# ==============================
# 8. 读取沿海社区 shapefile
# ==============================
shp_path = r'E:\coastline data\Coastline\coastline_Communities_GID02.shp'
gdf = gpd.read_file(shp_path)

gdf['GID_2'] = gdf['GID_2'].astype(str)
df_map['GID_2'] = df_map['GID_2'].astype(str)

gdf = gdf.merge(
    df_map[['GID_2', 'Subcontinental', 'Ineq_coef']],
    on='GID_2',
    how='left'
)

# ==============================
# 9. 背景图层
# ==============================
world = gpd.read_file(r'E:\coastline data\Coastline\coastline.shp')
south = gpd.read_file(r'D:\Gis Data\Coastline\South.shp')

# ==============================
# 10. 颜色规范（以 0 为中心）
# ==============================
vmin = gdf['Ineq_coef'].quantile(0.05)
vmax = gdf['Ineq_coef'].quantile(0.95)

norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

# ==============================
# 11. 绘制地图（横向图例置于下方）
# ==============================
from matplotlib import cm

fig, ax = plt.subplots(figsize=(14, 8))

# --- 世界底图 ---
world.plot(
    ax=ax,
    color='white',
    edgecolor='black',
    linewidth=0.4,
    zorder=1
)

# --- 沿海社区赋色 ---
gdf.plot(
    ax=ax,
    column='Ineq_coef',
    cmap='RdBu_r',
    norm=norm,
    linewidth=0,
    zorder=2
)

# --- 全球南方边界 ---
south.boundary.plot(
    ax=ax,
    linewidth=1.0,
    edgecolor='black',
    linestyle='--',
    zorder=3
)

ax.set_axis_off()
ax.set_title(
    'Subcontinental Infrastructure inequality effects on coastal flood risk',
    fontsize=16,
    pad=12
)

# ==============================
# 横向颜色图例（关键修改）
# ==============================
cax = fig.add_axes([0.25, 0.06, 0.5, 0.025])
# [left, bottom, width, height] —— 论文级比例

cb = plt.colorbar(
    cm.ScalarMappable(norm=norm, cmap='RdBu_r'),
    cax=cax,
    orientation='horizontal'
)

cb.set_label(
    'Effect of inequality on coastal flood risk (coef. from MixedLM)',
    fontsize=13,
    labelpad=6
)

cb.ax.tick_params(labelsize=11)
plt.tight_layout()
plt.show()




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "d:\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "d:\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "d:\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in s

AttributeError: _ARRAY_API not found

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject